In [1]:
# python modules
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import pyproj
import rasterio as rio
import sys

# local modules
sys.path.append("/home/nilscp/GIT/rastertools")
import utils

## Reading a window in a virtual mosaic layer
----

The idea here is to use a VRT (Virtual Dataset) that is a mosaic of the list of inputs (more info about the command can be found here https://gdal.org/programs/gdalbuildvrt.html). 

This is very useful for example for the Kaguya SLDEM2013 product (and all other products which consist of multiple tiles), where the product consist of a large amount of DEM products (1x1 degree tiles --> 360x120 = lot of files). I have generated a vrt file for all of those DEM tif files so that it can be easily queried for the area of interest. See the example below.

In [47]:
raster = Path('/tmp/test_vrt/test.vrt')
image = Path('/tmp/test_vrt/DTM_MAP_01_N00E000S01E001SC.tif') # a random image within the vrt 

In [40]:
with rio.open(raster) as src:
    meta = src.profile

In [41]:
meta

{'driver': 'VRT', 'dtype': 'int16', 'nodata': -32768.0, 'width': 8192, 'height': 8192, 'count': 1, 'crs': CRS.from_wkt('PROJCS["SIMPLE_CYLINDRICAL MOON",GEOGCS["GCS_MOON",DATUM["D_MOON",SPHEROID["MOON",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Equirectangular"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(7.4031617246699, 0.0, -3.70158086233495,
       0.0, -7.4031617246699, 30327.052005110247), 'blockxsize': 128, 'blockysize': 128, 'tiled': True}

In [42]:
meta['crs']

CRS.from_wkt('PROJCS["SIMPLE_CYLINDRICAL MOON",GEOGCS["GCS_MOON",DATUM["D_MOON",SPHEROID["MOON",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Equirectangular"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

### Specify the bounding box
In this vrt example, I have only data from 0.0-2.0 in longitude and -1.0 to 1.0 in latitude. Let's select a bounding box that will sample the four products contained in the virtual mosaic.

In [43]:
bbox = utils.bbox_world2crs(src, [0.25, -0.75, 1.75, 0.75]) #xmin, ymin, xmax, ymax

In [44]:
array = utils.read(raster, bbox)

In [45]:
array.shape

(6144, 6144, 1)

### Saving it to a raster
Can be easily save to a new raster (it uses a template raster, and the vrt raster can not be used as a template raster). You only have to select a random .tif within the vrt file (here defined by the variable image). And this is making the trick.

In [48]:
utils.to_raster(array, image, bbox, '/tmp/test_vrt/test.tif')